In [4]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from time import sleep
from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager 
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select  
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException 
import re

In [5]:
# llegamos a las actividades en Asturias
driver = webdriver.Chrome()             
url = 'https://www.civitatis.com/es/'
driver.get(url)
driver.maximize_window()  
driver.find_element('css selector','#didomi-notice-agree-button > span').click()
driver.find_element('css selector','#home-search-text').click()
sleep(5)
driver.find_element('css selector','#home-search-text').send_keys('asturias',Keys.TAB)
driver.find_element('css selector','#page-header__search-button').click()

In [6]:
pagina = 1
ciudad = 'asturias'


In [7]:
url = 'https://www.civitatis.com/es/{ciudad}/'          #url no dinamica de momento porque la pagina peta
                                                        #literalmente no hace la resq

In [8]:

def carga_datos():
    """_summary_:Cargar datos de actividades desde una página web.

    Realiza una solicitud HTTP a la URL especificada, analiza el contenido HTML
    para extraer información sobre actividades, incluyendo títulos, descripciones
    y precios, y devuelve esta información en un DataFrame de pandas.

    Returns:
        pd.DataFrame: Un DataFrame que contiene tres columnas: 'Actividad', 
        'Descripción' y 'Precio'. Si la solicitud falla o no se encuentran 
        productos, se devuelve un DataFrame vacío.

    """

    res = requests.get(url)
    if res.status_code == 200:
        print("Solicitud exitosa")
    else:
        print("Error en la solicitud")
        return pd.DataFrame()  

    sopa_actividades = BeautifulSoup(res.content, "html.parser")
    lista_productos = sopa_actividades.findAll("div", {'class': 'o-search-list__item'})
    # Listas para almacenar los datos
    titulos = []
    descripciones = []
    precios = []
    for producto in lista_productos:
        # Buscar los elementos
        titulo_elemento = producto.find('h2')
        descripcion_elemento = producto.find("div", {'class': 'comfort-card__text l-list-card__text'})
        precio_elemento = producto.find("span", {'class': 'comfort-card__price__text'})
        
        if titulo_elemento and descripcion_elemento and precio_elemento:
            titulo = titulo_elemento.get_text(strip=True)
            descripcion = descripcion_elemento.get_text(strip=True)
            precio = precio_elemento.get_text(strip=True)
            
            # Añadir los datos a las listas
            titulos.append(titulo)
            descripciones.append(descripcion)
            precios.append(precio)
      
    # Crear un DataFrame con los datos
    df = pd.DataFrame({
        'Actividad': titulos,
        'Descripción': descripciones,
        'Precio': precios,
    })

    return df

# Llamar a la función y mostrar el DataFrame
df_actividades = carga_datos()

df_actividades

Error en la solicitud


""


In [10]:
df_actividades.to_csv('Datos/actividades_asturias.csv')